<h2>Naive Bayes</h2>

In [1]:
from collections import defaultdict
import math
import re


def tokenize(msg):
    msg = msg.lower()
    all_words = re.findall("[a-z0-9]+",msg)
    return set(all_words)

def word_count(training_set):
    #training set consist of pairs (msg,is_spam)
    count = defaultdict(lambda: [0,0])
    for msg , is_spam in training_set:
        for word in tokenize(msg):
            count[word][0 if is_spam else 1]+=1
    return count

def word_probabilities(count,total_spam,total_non_spam,k=0.5):
    #k is to add bias to stop probabilty from being exactly 0
    #returns triplets w,p(w | spam) and p(w | ~spam)
    return [(w,(spam + k) / (total_spam + 2 * k),
 (non_spam + k) / (total_non_spam + 2 * k))
 for w, (spam, non_spam) in count.iteritems()]

def spam_probability(word_prob,msg):
    msg_words = tokenize(msg)
    log_prob_if_spam = log_prob_if_not_spam = 0.0

    for word, prob_if_spam, prob_if_not_spam in word_prob:
        if word in msg_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)
        prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [2]:
class NaiveBayesClassifier:
 def __init__(self, k=0.5):
    self.k = k
    self.word_probs = []
 def train(self, training_set):
    # count spam and non-spam messages
    num_spams = len([is_spam
        for message, is_spam in training_set
        if is_spam])
    num_non_spams = len(training_set) - num_spams
    # run training data through our "pipeline"
    word_counts = count_words(training_set)
    self.word_probs = word_probabilities(word_counts,
    num_spams,
    num_non_spams,
    self.k)
 def classify(self, message):
    return spam_probability(self.word_probs, message)